In [1]:
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer


csv = None
csv = pd.read_csv("Train_and_Validate_EEG.csv", header=0)

print(len(csv))

# Drop the blank columns
csv = csv.dropna(axis=1, how='all')

# convert sex to binary encoding
csv["sex"] = csv['sex'].replace(['F','M'],[-1,1])

# Mean Imputation
csv["IQ"] = csv["IQ"].fillna(csv["IQ"].mean())
csv["education"] = csv["education"].fillna(csv["education"].mean())

# Make the X data
all_columns = csv.columns
x_columns = all_columns.drop("ID").drop("main.disorder").drop("specific.disorder").drop("eeg.date")
X = csv[x_columns]

# make the Y data
y = csv["main.disorder"]

# IQ: normalize
X.loc[:,"IQ"] = (X["IQ"] - 100) / 15

# # Use Quantile Transformer
# quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0, n_quantiles=int(len(X)/5))

# for column_name in X.columns[4:]:
#    X.loc[:,column_name] = quantile_transformer.fit_transform(X[column_name].values.reshape(-1, 1)).flatten()

# Split train and test
X_train, X_test, y_train_, y_test_ = train_test_split(X, y, test_size=0.3, random_state=42)


scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train),columns=X.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns=X.columns)



852


<ipython-input-1-55860d0a5620>:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  csv["sex"] = csv['sex'].replace(['F','M'],[-1,1])
<ipython-input-1-55860d0a5620>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  csv["sex"] = csv['sex'].replace(['F','M'],[-1,1])
<ipython-input-1-55860d0a5620>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [16]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

# Create a RandomForestClassifier
model = RandomForestClassifier()

# modify y to take a number for each unique disorder
le = LabelEncoder()

# Fit the encoder to your data
y_train = y_train_.copy()
le.fit(y_train)
y_train = le.transform(y_train)

y_test = y_test_.copy()
y_test = le.transform(y_test)
# Fit the model
model.fit(X_train, y_train)

# Predict the output
y_pred = model.predict(X_test)
y_pred = pd.DataFrame(y_pred)

In [10]:
# Perform 5-fold cross-validation and obtain accuracy scores for each fold
scores = cross_val_score(model, X_test, y_test, cv=5)

# Compute the mean accuracy and the standard deviation
accuracy = scores.mean()
std_dev = scores.std()

print(accuracy, std_dev)

In [17]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Create a multioutput logistic regression model
model = MultiOutputClassifier(LogisticRegression(solver='lbfgs', max_iter=1000))

# modify y to take a number for each unique disorder
le = LabelEncoder()

# Fit the encoder to your data
y_train = y_train_.copy()

le.fit(y_train)
y_train = le.transform(y_train)

# Create an instance of OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the data
encoded_data = encoder.fit_transform(y_train.reshape(-1, 1))

# Convert the encoded data to a DataFrame
y_train = pd.DataFrame(encoded_data.toarray())

# Fit the model
model.fit(X_train, y_train)

# Predict the output
y_pred = model.predict(X_test)
y_pred = pd.DataFrame(y_pred)

In [32]:
# Perform 5-fold cross-validation and obtain accuracy scores for each fold
y_test = le.transform(y_test_)

# Fit and transform the data
encoded_data = encoder.fit_transform(y_test.reshape(-1, 1))

# Convert the encoded data to a DataFrame
y_test = pd.DataFrame(encoded_data.toarray())

scores = cross_val_score(model, X_test, y_test, cv=5)

# Compute the mean accuracy and the standard deviation
accuracy = scores.mean()
std_dev = scores.std()

print(accuracy, std_dev)

0.1055052790346908 0.026718979148211957


In [7]:
# XGBoost

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Create XGBoost classifier
clf = xgb.XGBClassifier(
    objective='multi:softmax',
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=100,
    tree_method="auto",
    early_stopping_rounds=10
)

# modify y to take a number for each unique disorder
le = LabelEncoder()

# Fit the encoder to your data
y_train = y_train_.copy()

le.fit(y_train)
y_train = le.transform(y_train)

y_test = y_test_.copy()
y_test = le.transform(y_test)



In [ ]:
scores = cross_val_score(model, X_test, y_test, cv=5)

# Compute the mean accuracy and the standard deviation
accuracy = scores.mean()
std_dev = scores.std()

print(accuracy, std_dev)

In [62]:
!pip install scikeras

In [4]:
# 1D CNN
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam


# Reshape data for 1D CNN (we need 3D input shape for Conv1D: [samples, timesteps, features])
X_train_cnn = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_cnn = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(len(y.unique()), activation='softmax'))  # For multi-class classification

cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
y_train = pd.get_dummies(y_train_)
y_test = pd.get_dummies(y_test_)
cnn_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32)

# Evaluate CNN model
cnn_val_loss, cnn_val_acc = cnn_model.evaluate(X_val_cnn, y_test)
print(f"1D CNN - Accuracy: {cnn_val_acc}")
print(f"1D CNN - Loss: {cnn_val_loss}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.1979 - loss: 1.9533
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.2614 - loss: 1.8815
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3131 - loss: 1.8242
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.3140 - loss: 1.7693
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.2996 - loss: 1.7606
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.3013 - loss: 1.7479
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.3099 - loss: 1.7355
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.3025 - loss: 1.7446
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3130 - loss: 1.7263
Epoch 10/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3208 - loss: 1.6836
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3387 - loss: 1.8143
1D CNN - Accuracy: 0.32421875
1D CNN - Loss: 1.8256103992462158


In [6]:
# Transformer Model
# Reshape the data for Transformer (similar to 1D CNN data)
X_train_transformer = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_transformer = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Instantiate and fit the encoder on the training labels
le = LabelEncoder()
y_train = y_train_.copy()
y_train_encoded = le.fit_transform(y_train)

y_test = y_test_.copy()
y_val_encoded = le.transform(y_test)

# Build the Transformer model
input_layer = Input(shape=(X_train_transformer.shape[1], 1))

# Transformer Block
x = LayerNormalization()(input_layer)
x = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
x = GlobalAveragePooling1D()(x)

# Dense Layers
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(y.unique()), activation='softmax')(x)  # Multi-class classification

transformer_model = Model(inputs=input_layer, outputs=output_layer)
transformer_model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the Transformer model
transformer_model.fit(X_train_transformer, y_train_encoded, epochs=10, batch_size=32)

# Evaluate Transformer model
transformer_val_loss, transformer_val_acc = transformer_model.evaluate(X_val_transformer, y_val_encoded)
print(f"Transformer - Accuracy: {transformer_val_acc}")
print(f"Transformer - Loss: {transformer_val_loss}")

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 92s 5s/step - accuracy: 0.2507 - loss: 1.9444
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 87s 5s/step - accuracy: 0.2951 - loss: 1.9384
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - accuracy: 0.2716 - loss: 1.9342
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 139s 4s/step - accuracy: 0.2698 - loss: 1.9283
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 145s 4s/step - accuracy: 0.3099 - loss: 1.9221
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 140s 4s/step - accuracy: 0.2918 - loss: 1.9176
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 144s 4s/step - accuracy: 0.2815 - loss: 1.9148
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - accuracy: 0.2762 - loss: 1.9104
Epoch 9/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 143s 4s/step - accuracy: 0.3026 - loss: 1.9038
Epoch 10/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 139s 4s/step - accuracy: 0.3003 - loss: 1.8987
8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.3200 - loss: 1.8914
Transformer - Accuracy: 0.30078125
Transformer - Loss: 1.8950459957122803
